In [103]:
%load_ext autoreload
%autoreload 2
from slack import SlackInteractor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
interactor = SlackInteractor()
data = interactor.fetch_new_messages()

Set conversations_oldest to 1722399936.894109
Loaded 18 old messages
Found 2 new messages
Updated complete_conversations.pkl with new messages


In [137]:
interactor.organize_threads(data)

[{'channel': 'general',
  'thread_ts': Timestamp('2024-07-31 23:08:19.263428926'),
  'messages': [{'ts': Timestamp('2024-07-31 23:08:19.263428926'),
    'user': 'Agentflow2',
    'text': 'Hello, World!',
    'is_bot': True},
   {'ts': Timestamp('2024-08-01 21:36:02.806229115'),
    'user': 'Eli.chen',
    'text': 'msg 2',
    'is_bot': False},
   {'ts': Timestamp('2024-08-01 21:36:04.156348944'),
    'user': 'Eli.chen',
    'text': 'msg 3',
    'is_bot': False},
   {'ts': Timestamp('2024-08-01 21:36:52.764959097'),
    'user': 'Eli.chen',
    'text': 'msg 4',
    'is_bot': False},
   {'ts': Timestamp('2024-08-01 21:44:57.960489035'),
    'user': 'Eli.chen',
    'text': 'msg 5',
    'is_bot': False},
   {'ts': Timestamp('2024-08-01 21:46:06.320808887'),
    'user': 'Eli.chen',
    'text': 'msg 6',
    'is_bot': False},
   {'ts': Timestamp('2024-08-01 21:47:40.985599041'),
    'user': 'Eli.chen',
    'text': 'msg 7',
    'is_bot': False},
   {'ts': Timestamp('2024-08-01 21:52:33.57442903

In [78]:
interactor.get_conversation_stats(data)

,max_len,total_length,total_messages,average_message_size
user_name,,,,


In [136]:
data

,type,subtype,ts,user,thread_ts,text,channel_id,text_clean,text_len,user_name,is_bot,channel_name
0,message,NaN,2024-08-01 21:53:12.864598989,U07EU2TQ6HG,2024-08-01 21:30:47.076678991,msg 1,C07EWH2GX7W,msg 1,5,Eli.chen,False,general
1,message,NaN,2024-08-01 21:53:01.920149088,U07EU2TQ6HG,2024-07-31 23:08:19.263428926,msg 9,C07EWH2GX7W,msg 9,5,Eli.chen,False,general


In [33]:
interactor.post_message('general', 'Hello, World!')